In [17]:
import requests
import datetime
import base64
import re
import pandas as pd
import traceback
import logging
import math

In [18]:
from config import client_id, client_secret

In [19]:
client_creds = f"{client_id}:{client_secret}"
type(client_creds)

str

In [20]:
client_creds_b64 = base64.b64encode(client_creds.encode())
type(client_creds_b64)

bytes

In [21]:
token_url = "https://accounts.spotify.com/api/token"
method = "POST"
token_data = {
    "grant_type": "client_credentials"
}
token_headers = {
    "Authorization": f"Basic {client_creds_b64.decode('utf-8')}",
    "Content-Type": 'application/x-www-form-urlencoded' # <base64 encoded client_id:client_secret>
}

In [22]:
r = requests.post(token_url, data=token_data, headers=token_headers)
json = r.json()
token = json['access_token']
valid_request = r.status_code in range(200, 299)

In [23]:
# Test if your access token has expired (it only lasts 60 minutes, 
# then you have to run cells to generate new one)
if valid_request:
    token_response_data = r.json()
    now = datetime.datetime.now()
    access_token = token_response_data['access_token']
    expires_in = token_response_data['expires_in'] # seconds
    expires = now + datetime.timedelta(seconds=expires_in)
    did_expire = expires < now

In [32]:
# Create three separate CSV/df files for each mood
 happy = ["0RH319xCjeU8VyTSqCF6M4"]
# sad = []

In [33]:
json = []

for playlist_id in demo_list:
    
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    print(url)
    playlist = getRequest(url, token)
    json.append(playlist)

json

https://api.spotify.com/v1/playlists/0RH319xCjeU8VyTSqCF6M4/tracks


[{'href': 'https://api.spotify.com/v1/playlists/0RH319xCjeU8VyTSqCF6M4/tracks?offset=0&limit=100',
  'items': [{'added_at': '2021-11-08T17:41:37Z',
    'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/qon9p4ro7d4cjfj1dsfw99k9z'},
     'href': 'https://api.spotify.com/v1/users/qon9p4ro7d4cjfj1dsfw99k9z',
     'id': 'qon9p4ro7d4cjfj1dsfw99k9z',
     'type': 'user',
     'uri': 'spotify:user:qon9p4ro7d4cjfj1dsfw99k9z'},
    'is_local': False,
    'primary_color': None,
    'track': {'album': {'album_type': 'single',
      'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/31TPClRtHm23RisEBtV3X7'},
        'href': 'https://api.spotify.com/v1/artists/31TPClRtHm23RisEBtV3X7',
        'id': '31TPClRtHm23RisEBtV3X7',
        'name': 'Justin Timberlake',
        'type': 'artist',
        'uri': 'spotify:artist:31TPClRtHm23RisEBtV3X7'}],
      'available_markets': ['AD',
       'AE',
       'AG',
       'AL',
       'AM',
       'AO',
       'AR',


In [10]:
# Function for getting request from Spotify
# You will have to edit the URL for querying playlist, artist, album, user info, etc.
def getRequest(url, token):
    data = requests.get(url, headers={"Accept": "application/json",
                                   "Content-Type": "application/json",
                                   "Authorization": f"Bearer {token}"})
    return data.json()

In [9]:
def getArtistPlaylistURIs(playlists_json):
    num_playlists = len(playlists_json['items'])
    URIs = []
    for i in range(num_playlists):
        URIs.append(playlists_json['items'][i]['uri'])
    return URIs

In [10]:
def getURI(uri_list):
#     p = re.compile(r'(?:spotify:playlist:)(\w{22})')
    p = re.compile(r'(?:spotify:(?:album|playlist|artist|track):)(\w{22})')
    URIs = [p.match(item).group(1) for item in uri_list]
    return URIs

In [11]:
# playlist has to be in the form of json data
def getSongInfo(playlist):
    length = len(playlist['items'])
    band_names = []
    band_ids = []
    song_names = []
    release_dates = []
    popularity = []
    uri_list = []
    for i in range(len(playlist['items'])):
        band_names.append(playlist['items'][i]['track']['artists'][0]['name'])
        band_ids.append(playlist['items'][i]['track']['artists'][0]['id'])
        song_names.append(playlist['items'][i]['track']['name'])
        release_dates.append(playlist['items'][i]['track']['album']['release_date'])
        popularity.append(playlist['items'][i]['track']['popularity'])
        uri_list.append(playlist['items'][i]['track']['uri'])
        
    data = {
        "band_name":band_names,
        "band_uri":band_ids,
        "song_name":song_names,
        "release_date": release_dates,
        "popularity":popularity,
        "track_uri": getURI(uri_list)
    }
    
    df = pd.DataFrame(data)
    return df

In [12]:
# Get the user's ID
user_id = '12121638402'

# Configure the URL
url = f"https://api.spotify.com/v1/users/{user_id}/playlists?offset=0&limit=50"

# Grab all the user's playlists (maximum of 50, unfortunately due to Spotify's APIs)
playlists = getRequest(url, token)

# Create a list of playlist's URIs 
playlist_URIs = getArtistPlaylistURIs(playlists)

# Filter this list on just the URIs
playlist_URIs_cleaned = getURI(playlist_URIs)
print(len(playlist_URIs_cleaned))

50


In [13]:
playlist_id = playlist_URIs_cleaned[0]
url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
playlist_json = getRequest(url, token)
df = getSongInfo(playlist_json)

In [14]:
playlist_URIs_cleaned.remove("4W6yjf9ChXo9lHHVH9FERe")
for playlist_id in playlist_URIs_cleaned:
    
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    playlist_json = getRequest(url, token)
    new_df = getSongInfo(playlist_json)
    
    try:
        df = df.append(new_df)
    except (TypeError, KeyError):
        print("'NoneType' object is not subscriptable")
   
df

,band_name,band_uri,song_name,release_date,popularity,track_uri
0,Slow Pulp,2JFTRDi5v7JtqoouVe1z5D,In Too Deep,2021-09-15,50,76rOZ9F8s8mX9N3NhZBgxt
1,Snail Mail,4QkSD9TRUnMtI8Fq1jXJJe,Valentine,2021-09-15,61,21brT2CZwKz8VPCf0nFxiP
2,Coco,4NxTtZTeJqQ1n27i2qnPCe,Knots,2021-08-24,25,2uDlO65509ESfCy4s07pYT
3,Kacey Musgraves,70kkdajctXSbqSMJbQO424,simple times,2021-09-10,69,1lNEXDlxVhsWaq2DLBUDgC
4,Kacey Musgraves,70kkdajctXSbqSMJbQO424,breadwinner,2021-09-10,76,2stbjvpFujerfDUuUUPX7o
...,...,...,...,...,...,...
95,Weezer,3jOstUTkEu2JkjvRdBA5Gu,El Scorcho,1996-09-24,0,1PB7gRWcvefzu7t3LJLUlf
96,The Goo Goo Dolls,2sil8z5kiy4r76CRTXxBCA,Name - New Version,2007-11-06,37,6WrP9bvTeNLalVEtLCgyhm
97,Green Day,7oPftvlwr6VrsViSDV7fJY,When I Come Around,1994-02-01,74,1Dr1fXbc2IxaK1Mu8P8Khz
98,Fall Out Boy,4UXqAaa6dQYAk18Lv7PEgX,"Sugar, We're Goin Down",2005-01-01,0,4zjFqN9fXAw91GNgJOCYX6


In [41]:
print(df.duplicated().sum())

896


In [17]:
track_json = getTrackInfo(df)

In [48]:
track_json[21]['audio_features'][99]

{'danceability': 0.627,
 'energy': 0.385,
 'key': 9,
 'loudness': -13.655,
 'mode': 0,
 'speechiness': 0.0537,
 'acousticness': 0.334,
 'instrumentalness': 0,
 'liveness': 0.182,
 'valence': 0.766,
 'tempo': 205.726,
 'type': 'audio_features',
 'id': '6a899bXlCuViXW2Qs8Rdqo',
 'uri': 'spotify:track:6a899bXlCuViXW2Qs8Rdqo',
 'track_href': 'https://api.spotify.com/v1/tracks/6a899bXlCuViXW2Qs8Rdqo',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6a899bXlCuViXW2Qs8Rdqo',
 'duration_ms': 230107,
 'time_signature': 4}

In [42]:
len(track_json)

23

In [57]:
tracks = getTrackInfo(df)

In [58]:
tracks

,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_md,time_signature
0,76rOZ9F8s8mX9N3NhZBgxt,0.629,0.794,9,-5.804,1,0.0760,0.014000,0.000043,0.1930,0.6290,107.944,221239,4
1,21brT2CZwKz8VPCf0nFxiP,0.488,0.623,0,-4.873,1,0.0317,0.000713,0.000000,0.1310,0.1710,129.874,196173,4
2,2uDlO65509ESfCy4s07pYT,0.513,0.340,5,-9.960,1,0.0295,0.918000,0.047300,0.1130,0.4460,107.947,189655,4
3,1lNEXDlxVhsWaq2DLBUDgC,0.654,0.713,1,-6.484,0,0.0846,0.628000,0.000000,0.0837,0.6940,80.043,167120,4
4,2stbjvpFujerfDUuUUPX7o,0.866,0.508,8,-8.591,0,0.0580,0.769000,0.000008,0.1100,0.4760,117.951,200840,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231,3NH2FJUSV3HGXZRtOc3Uwj,0.306,0.398,6,-12.767,1,0.0412,0.085700,0.000060,0.0756,0.0948,85.622,191828,4
2232,0ZjTS5zoXFR2w9QMRweDhs,0.646,0.623,8,-8.820,0,0.0337,0.274000,0.000000,0.0860,0.9780,120.651,219573,4
2233,63OozplRNF9pkljXOSWAof,0.369,0.473,9,-15.018,1,0.0329,0.073100,0.212000,0.0602,0.2400,87.907,250400,4
2234,6WrP9bvTeNLalVEtLCgyhm,0.429,0.868,9,-4.007,1,0.0364,0.004220,0.004750,0.2080,0.4660,73.985,252560,3


In [59]:
def getArtistJSON(df):   
    band_uris = list(df['band_uri'])
    uris = list(dict.fromkeys(band_uris))

    strings = []
    iterations = math.ceil(len(uris) / 50)
    for i in range(iterations):
        front_index = i*50
        end_index = front_index + 50
        uri_chunk = uris[front_index:end_index]
        joined_string = ",".join(uri_chunk)
        strings.append(joined_string)

    # joined_string = ",".join(uris)
    artists_json = []
    for i in strings:
        url = f"https://api.spotify.com/v1/artists?ids={i}"
        artists = getRequest(url, token)
        artists_json.append(artists)
    
    return artists_json

In [60]:
def artistDF(list_JSON):
    band_uris = []
    genres = []
    names = []
    followers = []
    for i in range(len(list_JSON)):
        num = len(list_JSON[i]['artists'])
        for j in range(num):
            genres.append(list_JSON[i]['artists'][j]['genres'])
            names.append(list_JSON[i]['artists'][j]['name'])
            band_uris.append(list_JSON[i]['artists'][j]['id'])
            followers.append(list_JSON[i]['artists'][j]['followers']['total'])
    
    data = {
        'band_uri': band_uris,
        'genres': genres,
        'names': names,
        'followers': followers
    }
    
    df = pd.DataFrame(data)
    return df

In [61]:
artists_json = getArtistJSON(df)

In [62]:
artists_df = artistDF(artists_json)

In [63]:
artists_df

,band_uri,genres,names,followers
0,2JFTRDi5v7JtqoouVe1z5D,"[bubblegrunge, chicago indie, indie garage roc...",Slow Pulp,50160
1,4QkSD9TRUnMtI8Fq1jXJJe,"[art pop, baltimore indie, bubblegrunge, indie...",Snail Mail,267030
2,4NxTtZTeJqQ1n27i2qnPCe,[],Coco,4733
3,70kkdajctXSbqSMJbQO424,"[contemporary country, country dawn, pop, texa...",Kacey Musgraves,1044695
4,5mFKYdmiYwNJTDtSzgFyQx,"[canadian indie folk, deep new americana, frea...",Andy Shauf,132849
...,...,...,...,...
859,0urTpYCsixqZwgNTkPJOJ4,"[dance pop, hip pop, pop, pop rap, r&b, urban ...",Aaliyah,2923027
860,4DD6xBZ1INIT3yGJRQLzej,"[alternative pop, alternative rock, gbvfi, pow...",Sugar,55545
861,6Hizgjo92FnMp8wGaRUNTn,"[neo mellow, pop rock, post-grunge]",Vertical Horizon,345030
862,40Yq4vzPs9VNUrIBG5Jr2i,"[alternative metal, alternative rock, grunge, ...",The Smashing Pumpkins,3574806


In [64]:
gabby_join = df.join(artists_df.set_index('band_uri'), on='band_uri')

,band_name,band_uri,song_name,release_date,popularity,track_uri,genres,names,followers
0,Slow Pulp,2JFTRDi5v7JtqoouVe1z5D,In Too Deep,2021-09-15,50,76rOZ9F8s8mX9N3NhZBgxt,"[bubblegrunge, chicago indie, indie garage roc...",Slow Pulp,50160
1,Snail Mail,4QkSD9TRUnMtI8Fq1jXJJe,Valentine,2021-09-15,61,21brT2CZwKz8VPCf0nFxiP,"[art pop, baltimore indie, bubblegrunge, indie...",Snail Mail,267030
2,Coco,4NxTtZTeJqQ1n27i2qnPCe,Knots,2021-08-24,25,2uDlO65509ESfCy4s07pYT,[],Coco,4733
3,Kacey Musgraves,70kkdajctXSbqSMJbQO424,simple times,2021-09-10,69,1lNEXDlxVhsWaq2DLBUDgC,"[contemporary country, country dawn, pop, texa...",Kacey Musgraves,1044695
4,Kacey Musgraves,70kkdajctXSbqSMJbQO424,breadwinner,2021-09-10,76,2stbjvpFujerfDUuUUPX7o,"[contemporary country, country dawn, pop, texa...",Kacey Musgraves,1044695
...,...,...,...,...,...,...,...,...,...
95,Weezer,3jOstUTkEu2JkjvRdBA5Gu,El Scorcho,1996-09-24,0,1PB7gRWcvefzu7t3LJLUlf,"[alternative rock, modern power pop, modern ro...",Weezer,2488874
96,The Goo Goo Dolls,2sil8z5kiy4r76CRTXxBCA,Name - New Version,2007-11-06,37,6WrP9bvTeNLalVEtLCgyhm,"[alternative rock, neo mellow, permanent wave,...",The Goo Goo Dolls,2047033
97,Green Day,7oPftvlwr6VrsViSDV7fJY,When I Come Around,1994-02-01,74,1Dr1fXbc2IxaK1Mu8P8Khz,"[permanent wave, punk]",Green Day,12713138
98,Fall Out Boy,4UXqAaa6dQYAk18Lv7PEgX,"Sugar, We're Goin Down",2005-01-01,0,4zjFqN9fXAw91GNgJOCYX6,"[emo, modern rock, rock]",Fall Out Boy,9121245


In [56]:
def getTrackInfo(df):
    track_uris = list(df['track_uri'])
    uris = list(dict.fromkeys(track_uris))
    
    strings = []
    iterations = math.ceil(len(uris) / 100)
    for i in range(iterations):
        front_index = i*100
        end_index = front_index + 100
        uri_chunk = uris[front_index:end_index]
        joined_string = ",".join(uri_chunk)
        strings.append(joined_string)

    track_json = []
    for i in strings:
        url = f"https://api.spotify.com/v1/audio-features?ids={i}"
        tracks = getRequest(url, token)
        track_json.append(tracks)
    
    danceability = []
    energy = []
    key = []
    loudness = []
    mode = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    track_id = []
    duration_ms = []
    time_signature = []
    
    # Loop through all of the JSON lists
    for i in range(len(track_json)):
        for j in range(len(track_json[i]['audio_features'])):
            track = track_json[i]
            audioFeature = track['audio_features'][j]
            danceability.append(audioFeature['danceability'])
            energy.append(audioFeature['energy'])
            key.append(audioFeature['key'])
            loudness.append(audioFeature['loudness']) 
            mode.append(audioFeature['mode']) 
            speechiness.append(audioFeature['speechiness'])
            acousticness.append(audioFeature['acousticness']) 
            instrumentalness.append(audioFeature['instrumentalness']) 
            liveness.append(audioFeature['liveness']) 
            valence.append(audioFeature['valence']) 
            tempo.append(audioFeature['tempo']) 
            track_id.append(audioFeature['id']) 
            duration_ms.append(audioFeature['duration_ms']) 
            time_signature.append(audioFeature['time_signature']) 
            
    data = {
        'track_uri': track_id,
        'danceability': danceability,
        'energy': energy,
        'key': key,
        'loudness':loudness,
        'mode':mode,
        "speechiness":speechiness,
        "acousticness":acousticness,
        "instrumentalness": instrumentalness,
        "liveness":liveness,
        "valence":valence,
        "tempo": tempo,
        "duration_md":duration_ms,
        "time_signature":time_signature
    }
    
    df = pd.DataFrame(data)
    return df
    

In [66]:
tracks.head()

,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_md,time_signature
0,76rOZ9F8s8mX9N3NhZBgxt,0.629,0.794,9,-5.804,1,0.0760,0.014000,0.000043,0.1930,0.629,107.944,221239,4
1,21brT2CZwKz8VPCf0nFxiP,0.488,0.623,0,-4.873,1,0.0317,0.000713,0.000000,0.1310,0.171,129.874,196173,4
2,2uDlO65509ESfCy4s07pYT,0.513,0.340,5,-9.960,1,0.0295,0.918000,0.047300,0.1130,0.446,107.947,189655,4
3,1lNEXDlxVhsWaq2DLBUDgC,0.654,0.713,1,-6.484,0,0.0846,0.628000,0.000000,0.0837,0.694,80.043,167120,4
4,2stbjvpFujerfDUuUUPX7o,0.866,0.508,8,-8.591,0,0.0580,0.769000,0.000008,0.1100,0.476,117.951,200840,4


In [72]:
gabby_join = gabby_join.set_index('track_uri')

In [74]:
tracks = tracks.set_index('track_uri')

In [77]:
gabby_join.merge(tracks, on='track_uri', how='left')
gabby_join.describe()

,popularity,followers,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_md,time_signature
count,3132.000000,3.132000e+03,3132.000000,3132.000000,3132.000000,3132.000000,3132.000000,3132.000000,3132.000000,3132.000000,3132.000000,3132.000000,3132.000000,3132.000000,3132.000000
mean,30.121328,8.572612e+05,0.542168,0.554243,5.226692,-9.305276,0.799808,0.042985,0.373527,0.164772,0.168817,0.469377,118.629706,236316.775223,3.906769
std,21.485089,2.718269e+06,0.147278,0.218821,3.495174,4.046931,0.400208,0.028490,0.327259,0.286815,0.134689,0.243968,28.570662,70621.528118,0.397524
min,0.000000,3.100000e+01,0.063000,0.003750,0.000000,-38.223000,0.000000,0.022400,0.000001,0.000000,0.007240,0.028900,37.114000,25316.000000,1.000000
25%,11.000000,2.403800e+04,0.444000,0.400000,2.000000,-11.080250,1.000000,0.029700,0.059375,0.000063,0.097200,0.267750,97.859250,192851.000000,4.000000
50%,30.000000,1.170850e+05,0.551000,0.570000,5.000000,-8.543000,1.000000,0.034650,0.305500,0.005115,0.117000,0.464000,116.002000,226091.000000,4.000000
75%,47.000000,7.848470e+05,0.642000,0.720000,8.000000,-6.671000,1.000000,0.044600,0.662000,0.175000,0.185000,0.646000,137.274000,269013.000000,4.000000
max,84.000000,4.479314e+07,0.952000,0.992000,11.000000,-1.380000,1.000000,0.325000,0.996000,0.979000,0.988000,0.985000,208.432000,969864.000000,5.000000


In [ ]:
def getUserPlaylist(user_id):
    # Configure the URL
    url = f"https://api.spotify.com/v1/users/{user_id}/playlists?offset=0&limit=50"

    # Grab all the user's playlists (maximum of 50, unfortunately due to Spotify's APIs)
    playlists = getRequest(url, token)

    # Create a list of playlist's URIs 
    playlist_URIs = getArtistPlaylistURIs(playlists)

    # Filter this list on just the URIs
    playlist_URIs_cleaned = getURI(playlist_URIs)
    print(len(playlist_URIs_cleaned))
    
    playlist_id = playlist_URIs_cleaned[0]
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    playlist_json = getRequest(url, token)
    df = getSongInfo(playlist_json)
    
#     playlist_URIs_cleaned.remove("4W6yjf9ChXo9lHHVH9FERe")
    for playlist_id in playlist_URIs_cleaned[1:]:
    
        url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
        playlist_json = getRequest(url, token)
        new_df = getSongInfo(playlist_json)

        try:
            df = df.append(new_df)
        except (TypeError, KeyError):
            print("'NoneType' object is not subscriptable")
   
    return df

In [ ]:
brigid_df = getUserPlaylist("bcemcphilamy")
brigid_artists_json = getArtistJSON(brigid_df)
brigid_artists_df = artistDF(brigid_artists_json)

In [ ]:
brigid_join = brigid_df.join(brigid_artists_df.set_index('band_uri'), on='band_uri')
brigid_join

In [ ]:
url = f"https://api.spotify.com/v1/me/top/artists"
top_tracks = getRequest(url, token)

In [ ]:
brigid_join.to_csv("spotify_brigid.csv", sep = ',')

In [ ]:
brigid_join['band_name'].value_counts()

In [ ]:
string = ['country world', 'rock', 'chicago indie']
test = [word.split(' ') for word in string]
test

for sublist in test: 
    print(sublist)

In [ ]:
# Get most common overarching genres
# 257 songs don't have assigned genres
overaching_genres = ['country','rock','pop','electronic','edm_dance','rnb','hiphop','kpop','latin','soul','jazz',
                    'classical']
genres = df_join['genres']
genres.iloc[0]

test = pd.DataFrame(item for item in genres)
# test = pd.DataFrame(genres.tolist())
# test[12].value_counts()
test

In [ ]:
top_genres = []
for col in test.columns:
    top_genres.append(test[col].value_counts().nlargest(20))

top_genres

In [ ]:
genre = ['rock','rock','pop']
df_test = pd.DataFrame({
    'genre': genre
})
print(df_test['genre'].value_counts())

In [ ]:
# # loop through all the lists in artists_json
# # then, loop through all of the artists within the list
# genres = artists_json[0]['artists'][0]['genres']
# name = artists_json[0]['artists'][0]['name']
# uri_id = artists_json[0]['artists'][0]['id']
# followers = artists_json[0]['artists'][0]['followers']['total']

In [ ]:
# len(artists_json[17]['artists'])

In [ ]:
# i = "5oOhM2DFWab8XhSdQiITry"
# url = f"https://api.spotify.com/v1/artists/{i}"
# artist_json = getRequest(url,token)
# genres = artist_json['genres']
# artist_json

In [ ]:

def getArtistGenre(artist_id_list):
    

In [ ]:
# For each band_uri in the dataframe, grab artists' stats like the genre, popularity, etc. 

In [ ]:
# get top 10 most frequent names
n = 10
df['band_name'].value_counts()[:n].index.tolist()

In [ ]:
df['popularity'].mean()

In [ ]:
# playlist_id = "4W6yjf9ChXo9lHHVH9FERe"
# url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
# playlist_json = getRequest(url, token)
# df
# try:
#     df = getSongInfo(playlist_json)
# except Exception as e:
#     logging.error(traceback.format_exc())
#     # Logs the error appropriately. 


In [ ]:
user_id = "bcemcphilamy"
url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
brigids_playlists = getRequest(url,token)
brigids_playlists

In [ ]:
# Madison Cunningham's playlists
user_id = "3cxtfy2f4gf5dqt194qfwgcx3"
url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
madison_cunningham = getRequest(url,token)
print("Madison Cunningham has: ", len(madison_cunningham['items']), "playlists.")

In [ ]:
# Madison Cunningham's songs in her playlist
def getArtistPlaylistURIs(playlists_json):
    num_playlists = len(playlists_json['items'])
    URIs = []
    for i in range(num_playlists):
        URIs.append(playlists_json['items'][i]['uri'])
    return URIs

mc_list = getArtistPlaylistURIs(madison_cunningham)
mc_list

In [ ]:
mc_list

In [ ]:
regex = re.compile('(?:spotify:playlist:)(\w{22})')
new_mc_list = list(filter(regex.match, mc_list))
new_mc_list

In [ ]:
list = ["guru99 get", "guru99 give", "guru Selenium"]
for element in list:
    z = re.match("^\w+", element)
if z:
    print((z.groups()))

In [ ]:
new_mc_list

In [ ]:
URIs = []
p = re.compile(r'(?:spotify:playlist:)(\w{22})')
for item in mc_list:
    m = p.match(item)
    uri = m.group(1)
    URIs.append(uri)
URIs

URIs = []
p = re.compile(r'(?:spotify:playlist:)(\w{22})')
URIs = [p.match(item).group(1) for item in mc_list]
# if m:
#     print('Match found: ', m.group(1))
# else:
#     print('No match')

In [ ]:
URIs = [p.match(item).group(1) for item in mc_list]
URIs

In [ ]:
import re

regex = r"(?:spotify:playlist:)(\w{22})"

test_str = ("spotify:playlist:5T4F9s5cizwJiI8EMNB3vv\n")

matches = re.finditer(regex, test_str, re.MULTILINE)

for matchNum, match in enumerate(matches, start=1):
    
    print ("Match {matchNum} was found at {start}-{end}: {match}".format(matchNum = matchNum, start = match.start(), end = match.end(), match = match.group()))
    
    for groupNum in range(0, len(match.groups())):
        groupNum = groupNum + 1
        
        print ("Group {groupNum} found at {start}-{end}: {group}".format(groupNum = groupNum, start = match.start(groupNum), end = match.end(groupNum), group = match.group(groupNum)))


In [ ]:
h = re.compile(r"(?:spotify:playlist:)(\w{22})")
h.match('spotify:playlist:5T4F9s5cizwJiI8EMNB3vv').group()

In [ ]:
user_id = "1237116029"
url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
laurels_playlists = getRequest(url,token)

In [ ]:
user_id = "1224941770"
url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
snail_mails_playlists = getRequest(url,token)

In [ ]:
user_id = "224sd2mespz4zevvqp4lyfxoq"
url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
phoebe_playlists = getRequest(url,token)

In [ ]:
playlist = phoebe_playlists['items'][0]['uri']

In [ ]:
# Example of how to use the function 

# Step 1. Grab the URI/id from Spotify (whether it's for a track, playlist, artist, etc.)
spotify_URI = '1EI0B66miJj5Fl408B7E9H'

# Step 2. Use an f string to create the url for the request. 
# See Spotify Console's endpoint for each of the data types you can pull from Spotify
# Sample documentation is here: https://developer.spotify.com/console/get-artist-albums/

url = f"https://api.spotify.com/v1/artists/{spotify_URI}/albums"

courtney_marie_andrews = getRequest(url,token)
courtney_marie_andrews

In [ ]:
courtney_marie_andrews = albums.json()
print(courtney_marie_andrews['items'][0]['name'])
print(courtney_marie_andrews['items'][0]['release_date'])
print(courtney_marie_andrews['items'][0]['total_tracks'])

In [ ]:
# https://api.spotify.com/v1/audio-analysis/{id}

# Example - pulling data for Michael Jackson's Billie Jean song
id = "5ChkMS8OtdzJeqyybCc9R5"
url = f"https://api.spotify.com/v1/audio-analysis/{id}"
billie_jean_song = getRequest(url,token)
billie_jean_song.keys()

In [ ]:
billie_jean_song

In [ ]:
# The curl is used to query things in git bash/terminal/command line
# curl -X "GET" "https://api.spotify.com/v1/artists//albums"
# -H "Accept: application/json" 
# -H "Content-Type: application/json" 
# -H "Authorization: Bearer "